In [2]:
import pandas as pd
import numpy as np
import pipes2 as pp

df = pd.read_csv('air_train.csv')
df = df.drop(['Unnamed: 0','id'], axis = 1)

df = pp.myPipe.transform(df)

X = df.drop(['satisfaction'], axis = 1)
y = df.satisfaction

X = X - X.mean()
X = X/X.std()


In [3]:
df = pd.read_csv('air_test.csv')
df = df.drop(['Unnamed: 0','id'], axis = 1)

df = pp.myPipe.transform(df)

X_t = df.drop(['satisfaction'], axis = 1)
y_t = df.satisfaction

X_t = X_t - X_t.mean()
X_t = X_t/X_t.std()


In [4]:
X.shape[1]

22

In [6]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()

model.add(layers.Dense(256, activation="relu", input_shape = (X.shape[1],),)) 
#model.add(layers.Dropout(0.4))

model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dense(256, activation="relu"))

model.add(layers.Dense(1, activation="sigmoid"))

# Компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
history = model.fit(X, y, validation_data = (X_t, y_t), 
                    epochs=150, batch_size=512)

Epoch 1/150
203/203 [==============================] - 2s 7ms/step - loss: 0.1937 - accuracy: 0.9207 - val_loss: 0.1314 - val_accuracy: 0.9448
Epoch 2/150
203/203 [==============================] - 1s 6ms/step - loss: 0.1179 - accuracy: 0.9508 - val_loss: 0.1092 - val_accuracy: 0.9556
Epoch 3/150
203/203 [==============================] - 1s 6ms/step - loss: 0.1016 - accuracy: 0.9575 - val_loss: 0.1018 - val_accuracy: 0.9559
Epoch 4/150
203/203 [==============================] - 1s 6ms/step - loss: 0.0949 - accuracy: 0.9590 - val_loss: 0.0998 - val_accuracy: 0.9556
Epoch 5/150
203/203 [==============================] - 1s 6ms/step - loss: 0.0885 - accuracy: 0.9619 - val_loss: 0.0958 - val_accuracy: 0.9597
Epoch 6/150
203/203 [==============================] - 1s 6ms/step - loss: 0.0842 - accuracy: 0.9636 - val_loss: 0.0966 - val_accuracy: 0.9587
Epoch 7/150
203/203 [==============================] - 1s 6ms/step - loss: 0.0816 - accuracy: 0.9644 - val_loss: 0.0928 - val_accuracy: 0.9593

In [44]:
from sklearn.metrics import confusion_matrix, f1_score

In [41]:
pr = model.predict(X_t)>0.5

812/812 [==============================] - 1s 724us/step


In [45]:
f1_score (y_t, pr)

0.9563199928612859

In [43]:
confusion_matrix(y_t, pr)

array([[14280,   293],
       [  686, 10717]])

In [ ]:
# кросс-валидация

In [64]:
from sklearn.model_selection import StratifiedKFold

In [65]:
def create_model():
    model = models.Sequential()

    model.add(layers.Dense(256, activation="relu", input_shape = (X.shape[1],), 
                       kernel_regularizer=tf.keras.regularizers.l2(0.002),)) 
                       #kernel_regularizer=tf.keras.regularizers.l2(0.002)))
        #model.add(layers.Dropout(0.4))

    model.add(layers.Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.002)))
    model.add(layers.Dense(256, activation="relu"))

    model.add(layers.Dense(1, activation="sigmoid"))

    # Компиляция модели
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="binary_crossentropy",
              metrics=["accuracy"])
    
    return model

In [89]:
def train_and_evaluate__model(model, data_train, labels_train, data_test, labels_test):
    
    history = model.fit(data_train, labels_train, validation_data = (data_test, labels_test), 
                        epochs=50, batch_size=512, verbose = 0)
    
    return history.history['val_accuracy'][-1]


In [90]:
kFold = StratifiedKFold(n_splits=5)
i = 0
for train, test in kFold.split(X, y):
    model = None
    model = create_model()
    i += 1
    v_a = train_and_evaluate__model(model, X[train], y[train], X[test], y[test])
    print ('split ',i, 'val_acc :', v_a)

split  1 val_acc : 0.9624657034873962
split  2 val_acc : 0.9595303535461426
split  3 val_acc : 0.9575573801994324
split  4 val_acc : 0.9636206030845642
split  5 val_acc : 0.963715136051178


In [80]:
X = np.asarray(X)

In [83]:
[train]

20579     1
20581     1
20584     1
20585     1
20590     1
         ..
103899    0
103900    1
103901    0
103902    0
103903    0
Name: satisfaction, Length: 83123, dtype: int64